In [ ]:
# pip install langchain chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery

# BigQuery QA

In [74]:
! pip install langchain chromadb sqlalchemy sqlalchemy-bigquery google-cloud-bigquery

In [6]:
import pandas as pd

from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

from langchain.agents import AgentExecutor
from langchain.llms import VertexAI

from google.cloud import aiplatform

### Set Variables

In [ ]:
service_account_file = "/path/to/your/file/secret.json" 

PROJECT_ID = "your-project-id"
DATASET = "IBM_ATTRITION"
TABLE = "ATTRITION_TABLE"
sqlalchemy_url = f'bigquery://{PROJECT_ID}/{DATASET}?credentials_path={service_account_file}'

### Load Data

In [22]:
df = pd.read_csv("IBM.csv")
df

,Age,Attrition,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany
0,41,Yes,Sales,1,2,Life Sciences,2,4,Single,5993,8,1,6
1,49,No,Research & Development,8,1,Life Sciences,3,2,Married,5130,1,3,10
2,37,Yes,Research & Development,2,2,Other,4,3,Single,2090,6,3,0
3,33,No,Research & Development,3,4,Life Sciences,4,3,Married,2909,1,3,8
4,27,No,Research & Development,2,1,Medical,1,2,Married,3468,9,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Research & Development,23,2,Medical,3,4,Married,2571,4,3,5
1466,39,No,Research & Development,6,1,Medical,4,1,Married,9991,4,3,7
1467,27,No,Research & Development,4,3,Life Sciences,2,2,Married,6142,1,3,6
1468,49,No,Sales,2,3,Medical,4,2,Married,5390,2,2,9


### LLM: Google PaLm & VertexAI

In [75]:
# initialize LLM

aiplatform.init(project=PROJECT_ID)

llm = VertexAI(model_name='text-bison@001')

In [76]:
# test LLM

question = "What day comes after Friday?"

llm(question)

'Saturday'

In [10]:
# Connect Sqlalchemy, LangChain, and VertexAI

db = SQLDatabase.from_uri(sqlalchemy_url)

llm = VertexAI(model_name='text-bison@001')

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(llm=llm,
                                    toolkit=toolkit,
                                    verbose=True,
                                    top_k=1000,
                                    )

/Users/sam/opt/anaconda3/envs/table_qa_env/lib/python3.11/site-packages/google/cloud/bigquery/client.py:570: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


### How many different Departments are there?

In [13]:
agent_executor.run("How many different Departments are there?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: ATTRITION_TABLE
Thought:I should query the schema of the ATTRITION_TABLE table to see what columns it has.
Action: sql_db_schema
Action Input: ATTRITION_TABLE
Observation: 
CREATE TABLE `ATTRITION_TABLE` (
	`Age` INT64, 
	`Attrition` BOOL, 
	`Department` STRING, 
	`DistanceFromHome` INT64, 
	`Education` INT64, 
	`EducationField` STRING, 
	`EnvironmentSatisfaction` INT64, 
	`JobSatisfaction` INT64, 
	`MaritalStatus` STRING, 
	`MonthlyIncome` INT64, 
	`NumCompaniesWorked` INT64, 
	`WorkLifeBalance` INT64, 
	`YearsAtCompany` INT64
)

/*
3 rows from ATTRITION_TABLE table:
Age	Attrition	Department	DistanceFromHome	Education	EducationField	EnvironmentSatisfaction	JobSatisfaction	MaritalStatus	MonthlyIncome	NumCompaniesWorked	WorkLifeBalance	YearsAtCompany
19	True	Sales	21	3	Other	4	2	Single	2121	1	4	1
21	True	Research & Development	18	1	Other	4	4	Single	2693	1	2	1
22	False	Research & Development	26

'There are 3 different Departments.'

In [24]:
df['Department'].nunique()

3

### What's the average monthly income for the Sales department?

In [20]:
agent_executor.run("What's the average monthly income for the Sales department?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: ATTRITION_TABLE
Thought:I don't know what the ATTRITION_TABLE is. I should ask for a schema of it.
Action: sql_db_schema
Action Input: ATTRITION_TABLE
Observation: 
CREATE TABLE `ATTRITION_TABLE` (
	`Age` INT64, 
	`Attrition` BOOL, 
	`Department` STRING, 
	`DistanceFromHome` INT64, 
	`Education` INT64, 
	`EducationField` STRING, 
	`EnvironmentSatisfaction` INT64, 
	`JobSatisfaction` INT64, 
	`MaritalStatus` STRING, 
	`MonthlyIncome` INT64, 
	`NumCompaniesWorked` INT64, 
	`WorkLifeBalance` INT64, 
	`YearsAtCompany` INT64
)

/*
3 rows from ATTRITION_TABLE table:
Age	Attrition	Department	DistanceFromHome	Education	EducationField	EnvironmentSatisfaction	JobSatisfaction	MaritalStatus	MonthlyIncome	NumCompaniesWorked	WorkLifeBalance	YearsAtCompany
19	True	Sales	21	3	Other	4	2	Single	2121	1	4	1
21	True	Research & Development	18	1	Other	4	4	Single	2693	1	2	1
22	False	Research & Development	26	2	Other

'The average monthly income for the Sales department is 6959.17.'

In [28]:
df[['Department', 'MonthlyIncome']].groupby('Department').mean().round(2).reset_index()

,Department,MonthlyIncome
0,Human Resources,6654.51
1,Research & Development,6281.25
2,Sales,6959.17


### Which department has the highest number of attrition and how far on average is the distance from home for those who attited from this department?

In [44]:
agent_executor.run("Which department has the highest number of attrition and how far on average is the distance from home for those who attited from this department?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: ATTRITION_TABLE
Thought:I should query the schema of the ATTRITION_TABLE table to see what columns it has.
Action: sql_db_schema
Action Input: ATTRITION_TABLE

Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..



Observation: 
CREATE TABLE `ATTRITION_TABLE` (
	`Age` INT64, 
	`Attrition` BOOL, 
	`Department` STRING, 
	`DistanceFromHome` INT64, 
	`Education` INT64, 
	`EducationField` STRING, 
	`EnvironmentSatisfaction` INT64, 
	`JobSatisfaction` INT64, 
	`MaritalStatus` STRING, 
	`MonthlyIncome` INT64, 
	`NumCompaniesWorked` INT64, 
	`WorkLifeBalance` INT64, 
	`YearsAtCompany` INT64
)

/*
3 rows from ATTRITION_TABLE table:
Age	Attrition	Department	DistanceFromHome	Education	EducationField	EnvironmentSatisfaction	JobSatisfaction	MaritalStatus	MonthlyIncome	NumCompaniesWorked	WorkLifeBalance	YearsAtCompany
19	True	Sales	21	3	Other	4	2	Single	2121	1	4	1
21	True	Research & Development	18	1	Other	4	4	Single	2693	1	2	1
22	False	Research & Development	26	2	Other	2	3	Married	2814	1	2	4
*/
Thought:

Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..


I should query the ATTRITION_TABLE table to find the department with the highest number of attrition.
Action: sql_db_query
Action Input: SELECT Department, AVG(DistanceFromHome) FROM ATTRITION_TABLE WHERE Attrition = True GROUP BY Department ORDER BY COUNT(*) DESC LIMIT 1
Observation: [('Research & Development', 10.263157894736842)]
Thought:I now know the final answer
Final Answer: The department with the highest number of attrition is Research & Development, and the average distance from home for those who attited from this department is 10.263157894736842.

> Finished chain.


'The department with the highest number of attrition is Research & Development, and the average distance from home for those who attited from this department is 10.263157894736842.'

In [42]:
df[['Attrition','Department', 'DistanceFromHome']].groupby(['Department','Attrition']).count().reset_index()

,Department,Attrition,DistanceFromHome
0,Human Resources,No,51
1,Human Resources,Yes,12
2,Research & Development,No,828
3,Research & Development,Yes,133
4,Sales,No,354
5,Sales,Yes,92


In [41]:
df[['Attrition','Department', 'DistanceFromHome']].groupby(['Department','Attrition']).mean().round(2).reset_index()

,Department,Attrition,DistanceFromHome
0,Human Resources,No,7.59
1,Human Resources,Yes,13.42
2,Research & Development,No,8.96
3,Research & Development,Yes,10.26
4,Sales,No,8.99
5,Sales,Yes,10.80


### What is the percentage of employees who live more than 5 miles from home and who have attrited from the Sales department?

In [77]:
agent_executor.run("What is the percentage of employees who live more than 5 miles from home and who have attrited from the Sales department?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: ATTRITION_TABLE
Thought:I should look at the schema of the ATTRITION_TABLE to see what columns I can query.
Action: sql_db_schema
Action Input: ATTRITION_TABLE
Observation: 
CREATE TABLE `ATTRITION_TABLE` (
	`Age` INT64, 
	`Attrition` BOOL, 
	`Department` STRING, 
	`DistanceFromHome` INT64, 
	`Education` INT64, 
	`EducationField` STRING, 
	`EnvironmentSatisfaction` INT64, 
	`JobSatisfaction` INT64, 
	`MaritalStatus` STRING, 
	`MonthlyIncome` INT64, 
	`NumCompaniesWorked` INT64, 
	`WorkLifeBalance` INT64, 
	`YearsAtCompany` INT64
)

/*
3 rows from ATTRITION_TABLE table:
Age	Attrition	Department	DistanceFromHome	Education	EducationField	EnvironmentSatisfaction	JobSatisfaction	MaritalStatus	MonthlyIncome	NumCompaniesWorked	WorkLifeBalance	YearsAtCompany
19	True	Sales	21	3	Other	4	2	Single	2121	1	4	1
21	True	Research & Development	18	1	Other	4	4	Single	2693	1	2	1
22	False	Research & Development	2

'0.5762331838565022'

In [65]:
df[(df['Department']=='Sales')&(df['DistanceFromHome']>5)]['DistanceFromHome'].value_counts().sum() / \
df[(df['Department']=='Sales')]['DistanceFromHome'].value_counts().sum()

0.5762331838565022

In [63]:
df[(df['Department']=='Sales')&(df['DistanceFromHome'] > 5)]['Attrition'].value_counts()

Attrition
No     199
Yes     58
Name: count, dtype: int64

In [58]:

df[(df['Department']=='Sales')&(df['Attrition']=='Yes')&(df['DistanceFromHome']>5)]['DistanceFromHome'].value_counts().sum() / \
df[(df['Department']=='Sales')&(df['Attrition']=='Yes')]['DistanceFromHome'].value_counts().sum()

0.6304347826086957

### What department has the most Attrition?

In [72]:
agent_executor.run("What department has the most Attrition?")



> Entering new AgentExecutor chain...


Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..


Action: sql_db_list_tables
Action Input: 
Observation: ATTRITION_TABLE
Thought:

Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..


I should query the schema of the ATTRITION_TABLE table to see what columns I can use in my query.
Action: sql_db_schema
Action Input: ATTRITION_TABLE

/Users/sam/opt/anaconda3/envs/table_qa_env/lib/python3.11/site-packages/google/cloud/bigquery/client.py:570: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(
Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..



Observation: 
CREATE TABLE `ATTRITION_TABLE` (
	`Age` INT64, 
	`Attrition` BOOL, 
	`Department` STRING, 
	`DistanceFromHome` INT64, 
	`Education` INT64, 
	`EducationField` STRING, 
	`EnvironmentSatisfaction` INT64, 
	`JobSatisfaction` INT64, 
	`MaritalStatus` STRING, 
	`MonthlyIncome` INT64, 
	`NumCompaniesWorked` INT64, 
	`WorkLifeBalance` INT64, 
	`YearsAtCompany` INT64
)

/*
3 rows from ATTRITION_TABLE table:
Age	Attrition	Department	DistanceFromHome	Education	EducationField	EnvironmentSatisfaction	JobSatisfaction	MaritalStatus	MonthlyIncome	NumCompaniesWorked	WorkLifeBalance	YearsAtCompany
19	True	Sales	21	3	Other	4	2	Single	2121	1	4	1
21	True	Research & Development	18	1	Other	4	4	Single	2693	1	2	1
22	False	Research & Development	26	2	Other	2	3	Married	2814	1	2	4
*/
Thought:

Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..
Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..


I should write a query to get the department with the most Attrition.
Action: sql_db_query
Action Input: SELECT Department, count(*) as AttritionCount FROM ATTRITION_TABLE GROUP BY Department ORDER BY AttritionCount DESC LIMIT 1
Observation: [('Research & Development', 961)]
Thought:I now know the final answer
Final Answer: Research & Development has the most Attrition.

> Finished chain.


'Research & Development has the most Attrition.'

In [82]:
df.groupby(['Department', 'Attrition']).count().reset_index().rename(columns = {'Age':'Count'})[['Department', 'Attrition', 'Count']]

,Department,Attrition,Count
0,Human Resources,No,51
1,Human Resources,Yes,12
2,Research & Development,No,828
3,Research & Development,Yes,133
4,Sales,No,354
5,Sales,Yes,92
